In [ ]:
import numpy as np
from numpy import linalg
from numba import jit
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.transforms as mtransforms
from matplotlib.gridspec import SubplotSpec
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

from scipy.constants import pi
from scipy.interpolate import RectBivariateSpline, griddata

import os
import ast
import pandas as pd

def set_pandas_display_options() -> None:
    """Set pandas display options."""
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = None
    display.max_rows = None
    display.max_colwidth = None
    display.width = None
    display.chop_threshold = None
    display.precision = 14  # set as needed

set_pandas_display_options()

# Define methods

In [ ]:
def mask_singularities(data,cutoff = 2):
    '''
    If data points have a value bigger than the cutoff, it is replaced by the average value of their neighbours
    '''
    for i, col in enumerate(data):
        for j, e in enumerate(col):
            if abs(e)>cutoff:
                if i == 0:
                    if j == 0:
                        data[i][j] = np.average([data[i][j+1],
                                                 data[i+1][j],data[i+1][j+1]])
                        continue
                    elif j == len(col)-1:
                        data[i][j] = np.average([data[i][j-1],
                                                 data[i+1][j-1],data[i+1][j]])
                        continue
                    else:
                        data[i][j] = np.average([data[i][j-1],data[i][j+1],
                                                 data[i+1][j-1],data[i+1][j],data[i+1][j+1]])

                elif i == len(data)-1:
                    if j == 0:
                        data[i][j] = np.average([data[i-1][j],data[i-1][j+1],
                                                 data[i][j+1]])
                        continue
                    elif j == len(col)-1:
                        data[i][j] = np.average([data[i-1][j-1],data[i-1][j],
                                                 data[i][j-1]])
                    else:
                        data[i][j] = np.average([data[i-1][j-1],data[i-1][j],data[i-1][j+1],
                                                 data[i][j-1],data[i][j+1]])
                elif j == 0:
                    data[i][j] = np.average([data[i-1][j],data[i-1][j+1],
                                             data[i][j+1],
                                             data[i+1][j],data[i+1][j+1]])
                elif j == len(col)-1:
                    data[i][j] = np.average([data[i-1][j-1],data[i-1][j],
                                             data[i][j-1],
                                             data[i+1][j-1],data[i+1][j]])
                else:
                    data[i][j] = np.average([data[i-1][j-1],data[i-1][j],data[i-1][j+1],
                                             data[i][j-1],data[i][j+1],
                                             data[i+1][j-1],data[i+1][j],data[i+1][j+1]])
    return data

def mirror_quadrant(data,n_shift,n_mag,odd=True):
    """
    mirror first quadrant
    """
    z = np.zeros((2*n_mag-1, 2*n_shift-1))
    for i in range(2*n_mag):
        for j in range(2*n_shift):
            if i < n_mag and j < n_shift:
                z[i, j] = (-1)**odd * data[n_mag-1-i, n_shift-1-j]
            elif i > n_mag and j < n_shift:
                z[i-1, j] = data[i-n_mag, j]
            elif i < n_mag and j > n_shift:
                z[i, j-1] = (-1)**odd * data[n_mag-1-i, j-n_shift]
            elif i > n_mag and j > n_shift:
                z[i-1, j-1] = data[i-n_mag, 2*n_shift-1-j]
    return z

# Import data

## Read files for parameters

In [ ]:
files = [x[0] for x in os.walk('./data/')][1::]
df = pd.DataFrame()
for file in files:
    data = pd.read_json(file+'/params.json',orient='index').transpose()
    data["key"] = file[7:]
    if "n_shift" in data:
        if "n_mag" in data:
            df = pd.concat([df,data])

In [ ]:
display(df.set_index('key').sort_index(ascending=False))

## Select data

In [ ]:
key = '1700836922'

## Import file and extract data

In [ ]:
outdir = './data/' + key

# parameter
tex, size = df.set_index('key').at[key,'texture'], df.set_index('key').at[key,'system_sizes'],
t, m = df.set_index('key').at[key,'t'], df.set_index('key').at[key,'m'],
theta, ids = df.set_index('key').at[key,'q'], df.set_index('key').at[key,'ids']
n_shift, min_shift, max_shift = df.set_index('key').at[key,'n_shift'], df.set_index('key').at[key,'min_shift'], df.set_index('key').at[key,'max_shift']
n_mag, min_mag, max_mag = df.set_index('key').at[key,'n_mag'], df.set_index('key').at[key,'min_mag'], df.set_index('key').at[key,'max_mag']
save_spec, tau1tau2, u1u2, tau1tau2u1u2 = df.set_index('key').at[key,'save_spec'], df.set_index('key').at[key,'tau1tau2'], df.set_index('key').at[key,'u1u2'], df.set_index('key').at[key,'tau1tau2u1u2']
phi, eta = df.set_index('key').at[key,'phi'], df.set_index('key').at[key,'eta']

# data
shifts = np.linspace(min_shift,max_shift,n_shift)
mags = np.linspace(min_mag,max_mag,n_mag)
    
fermi = np.load(outdir+"/fermi.npy").real.reshape((n_mag,n_shift))
ch_tau1tau2 = np.load(outdir+"/ch_tau1tau2.npy").real.reshape((n_mag,n_shift))
ch_u1u2 = np.load(outdir+"/ch_u1u2.npy").real.reshape((n_mag,n_shift))
ch_tau1tau2u1u2 = np.load(outdir+"/ch_tau1tau2u1u2.npy").real.reshape((n_mag,n_shift))

In [ ]:
fig, ax = plt.subplots(2,2)
ax[0,0].imshow(fermi,interpolation='nearest',cmap='inferno')
ax[0,1].imshow(ch_tau1tau2,vmin=-2,vmax=2,interpolation='nearest',cmap='RdBu')
ax[1,0].imshow(ch_u1u2,vmin=-2,vmax=2,interpolation='nearest',cmap='RdBu')
ax[1,1].imshow(ch_tau1tau2u1u2,vmin=-2,vmax=2,interpolation='nearest',cmap='RdBu')

## Preprocessing

In [ ]:
# mask_singularities(ch_tau1tau2,1);
# mask_singularities(ch_u1u2,2);
# mask_singularities(ch_tau1tau2u1u2,1);

In [ ]:
shifts = np.linspace(0,2*np.pi,2*n_shift-1)
mags = np.linspace(-2,2,2*n_mag-1)

fermi = mirror_quadrant(fermi,n_shift,n_mag,False)
ch_tau1tau2 = mirror_quadrant(ch_tau1tau2,n_shift,n_mag,True)
ch_u1u2 = mirror_quadrant(ch_u1u2,n_shift,n_mag,True)
ch_tau1tau2u1u2 = mirror_quadrant(ch_tau1tau2u1u2,n_shift,n_mag,False)

# Plots

In [ ]:
# Plot parameter
mpl.pyplot.rcdefaults()
plt.rcParams['figure.figsize'] = [20,18]
plt.rcParams['savefig.facecolor'] = "white"
mpl.rcParams['figure.dpi'] = 300
mpl.rcParams['axes.linewidth'] = 1.2
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'

tfs    = 40 #title font size
lfs    = 40 #label font size
fs     = 40 #font size
cbarfs = 18 #colorbar font size

shade = 'auto'
# shade = 'gouraud'

### 2x2

In [ ]:
X, Y = np.meshgrid(mags,shifts,indexing="ij")

fig = plt.figure()

gs = fig.add_gridspec(2,2, hspace=0,wspace=0)
ax2 = plt.subplot(gs.new_subplotspec((1, 0), rowspan=1))
ax3 = plt.subplot(gs.new_subplotspec((1, 1), rowspan=1),sharey=ax2)
ax0 = plt.subplot(gs.new_subplotspec((0, 0), rowspan=1),sharex=ax2)
ax1 = plt.subplot(gs.new_subplotspec((0, 1), rowspan=1),sharey=ax0,sharex=ax3)

col0 = ax0.pcolormesh(X, Y, fermi, cmap=plt.cm.inferno,shading=shade,snap=True)
ax0.set_title(r'$E_F$',fontsize=tfs, x=0.93, y=0.93, loc='left',color='white',backgroundcolor= 'black')
ax0.set_xlabel(r'$m$',fontsize=lfs)
ax0.set_ylabel(r'$\varphi$',fontsize=lfs)
cbar0 = plt.colorbar(col0,ax=ax1,shrink=0.95)
ax0.get_xaxis().set_visible(False)

col1 = ax1.pcolormesh(X, Y, ch_tau1tau2, vmin=-2.2,vmax=2.2, cmap=plt.cm.RdBu,shading=shade,snap=True)
ax1.set_title(r'$Ch_{t_1 t_2}$',fontsize=tfs, x=0.83, y=0.93, loc='left')
ax1.set_xlabel(r'$m$',fontsize=lfs)
ax1.set_ylabel(r'$\varphi$',fontsize=lfs)
cbar1 = plt.colorbar(col1,ax=ax3,shrink=0.95,extend='both')
cbar1.cmap.set_over('black')
cbar1.cmap.set_under('black')
ax1.get_xaxis().set_visible(False)
ax1.get_yaxis().set_visible(False)

col2 = ax2.pcolormesh(X, Y, ch_u1u2, vmin=-2.2,vmax=2.2, cmap=plt.cm.RdBu,shading=shade,snap=True)
ax2.set_title(r'$Ch_{u_1 u_2}$',fontsize=tfs, x=0.84, y=0.93, loc='left')
ax2.set_xlabel(r'$m$',fontsize=lfs)
ax2.set_ylabel(r'$\varphi$',fontsize=lfs)
# plt.colorbar(col2,ax=ax2,shrink=0.75)

col3 = ax3.pcolormesh(X, Y, ch_tau1tau2u1u2, vmin=-2.2,vmax=2.2, cmap=plt.cm.RdBu,shading=shade,snap=True)
ax3.set_title(r'$Ch_{t_1 t_2 u_1 u_2}$',fontsize=tfs, x=0.73, y=0.92, loc='left')
ax3.set_xlabel(r'$m$',fontsize=lfs)
ax3.set_ylabel(r'$\varphi$',fontsize=lfs)
ax3.get_yaxis().set_visible(False)
# plt.colorbar(col3,ax=ax3,shrink=0.75)

ax2.set_xticks(np.linspace(-2,2,4,endpoint=False))
ax2.xaxis.set_minor_locator(AutoMinorLocator())
ax3.set_xticks(np.linspace(-2,2,5))
ax3.xaxis.set_minor_locator(AutoMinorLocator())

labels = [r'${:.2f} \pi$'.format(n) for n in np.linspace(0,2,5)]

ax0.set_yticks(np.linspace(0,2*np.pi,5))
ax0.yaxis.set_minor_locator(AutoMinorLocator())
ax0.set_yticklabels(labels,fontsize=fs)

labels = [r'${:.2f} \pi$'.format(n) for n in np.linspace(0,2,4,endpoint=False)]
ax2.set_yticks(np.linspace(0,2*np.pi,4,endpoint=False))
ax2.yaxis.set_minor_locator(AutoMinorLocator())
ax2.set_yticklabels(labels,fontsize=fs)

for ax in fig.get_axes():
    ax.tick_params(axis='both', which='major', labelsize=fs)
    ax.hlines(np.pi, -2, 2, color='lightgray', alpha=0.3)
    ax.vlines(0, 0, 2*np.pi, color='lightgray', alpha=0.3)

fig.tight_layout()
plt.savefig("./Plots/ChernDiagram2x2.png", dpi=300, bbox_inches = 'tight')
plt.savefig("./Plots/lowres/ChernDiagram2x2.png", dpi=100, bbox_inches = 'tight')

### 1x3

In [ ]:
X, Y = np.meshgrid(mags,shifts,indexing="ij")

fig = plt.figure(figsize=(20,8))

gs = fig.add_gridspec(1,3, hspace=0,wspace=0)
ax1 = plt.subplot(gs.new_subplotspec((0,0)))
ax2 = plt.subplot(gs.new_subplotspec((0,1), rowspan=1),sharey=ax1)
ax3 = plt.subplot(gs.new_subplotspec((0,2), rowspan=1),sharey=ax1)
# ax0 = plt.subplot(gs.new_subplotspec((0, 0), rowspan=1),sharex=ax2)

# col0 = ax0.pcolormesh(X, Y, fermi, cmap='inferno',shading=shade)
# ax0.set_title(r'$E_F$',fontsize=tfs, x=0.95, y=0.95, loc='left',color='white',backgroundcolor= 'black')
# ax0.set_xlabel(r'$m$',fontsize=lfs)
# ax0.set_ylabel(r'$\varphi$',fontsize=lfs)
# plt.colorbar(col0,ax=ax1,shrink=0.95)
# ax0.get_xaxis().set_visible(False)

col1 = ax1.pcolormesh(X, Y, ch_tau1tau2, vmin=-2.1,vmax=2.1, cmap=plt.cm.RdBu,shading=shade,snap=True)
ax1.set_title(r'$Ch_{t_1 t_2}$',fontsize=tfs, x=0.80, y=0.92, loc='left')
ax1.set_xlabel(r'$m$',fontsize=lfs)
ax1.set_ylabel(r'$\varphi$',fontsize=lfs)
# ax1.get_xaxis().set_visible(False)
# ax1.get_yaxis().set_visible(False)

col2 = ax2.pcolormesh(X, Y, ch_u1u2, vmin=-2.1,vmax=2.1, cmap=plt.cm.RdBu,shading=shade,snap=True)
ax2.set_title(r'$Ch_{u_1 u_2}$',fontsize=tfs, x=0.77, y=0.92, loc='left')
ax2.set_xlabel(r'$m$',fontsize=lfs)
ax2.set_ylabel(r'$\varphi$',fontsize=lfs)
ax2.get_yaxis().set_visible(False)
# plt.colorbar(col2,ax=ax2,shrink=0.75)

col3 = ax3.pcolormesh(X, Y, ch_tau1tau2u1u2, vmin=-2.1,vmax=2.1, cmap=plt.cm.RdBu,shading=shade,snap=True)
ax3.set_title(r'$Ch_{t_1 t_2 u_1 u_2}$',fontsize=tfs, x=0.68, y=0.92, loc='left')
ax3.set_xlabel(r'$m$',fontsize=lfs)
ax3.set_ylabel(r'$\varphi$',fontsize=lfs)
ax3.get_yaxis().set_visible(False)
# plt.colorbar(col3,ax=ax3,shrink=0.75)

cbar1 = plt.colorbar(col1,ax=fig.get_axes(),anchor=(2.2,0.5),extend='both')
cbar1.cmap.set_over('black')
cbar1.cmap.set_under('black')

ax1.set_xticks(np.linspace(-2,2,4,endpoint=False))
ax1.xaxis.set_minor_locator(AutoMinorLocator())
ax2.set_xticks(np.linspace(-2,2,4,endpoint=False))
ax2.xaxis.set_minor_locator(AutoMinorLocator())
ax3.set_xticks(np.linspace(-2,2,5))
ax3.xaxis.set_minor_locator(AutoMinorLocator())

labels = [r'${:.1f} \pi$'.format(n) for n in np.linspace(0,2,5)]

ax1.set_yticks(np.linspace(0,2*np.pi,5))
ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.set_yticklabels(labels,fontsize=fs)

# labels = [r'${:.1f} \pi$'.format(n) for n in np.linspace(min_shift/np.pi,max_shift/np.pi,10,endpoint=False)]
# ax2.set_yticks(np.linspace(min_shift,max_shift,10,endpoint=False))
# ax2.yaxis.set_minor_locator(AutoMinorLocator())
# ax2.set_yticklabels(labels,fontsize=fs)

for ax in fig.get_axes():
    ax.tick_params(axis='both', which='major', labelsize=fs)
    ax.hlines(np.pi, -2, 2, color='lightgray', alpha=0.3)
    ax.vlines(0, 0, 2*np.pi, color='lightgray', alpha=0.3)

fig.tight_layout()
plt.savefig("./Plots/ChernDiagram3.png", dpi=300, bbox_inches = 'tight')
plt.savefig("./Plots/lowres/ChernDiagram3.png", dpi=100, bbox_inches = 'tight')

# Analytic winding number diagram

In [ ]:
net_mag1 = np.linspace(-2,np.sqrt(3)/2,10000,endpoint=False)
net_mag2 = np.linspace(np.sqrt(3)/2+0.00001,2,10000)

def winding_boundary(m):
    s1 = ( 3*(np.pi + np.arccos(m/np.sqrt(3))) )%(2*np.pi)
    s2 = ( 3*(np.pi - np.arccos(m/np.sqrt(3))) )%(2*np.pi)
    s3 = ( 2*np.pi - np.arccos(m*np.sqrt(3)) )%(2*np.pi)
    s4 = ( 4*np.pi + np.arccos(m*np.sqrt(3)) )%(2*np.pi)
    return np.array([s1,s2,s3,s4])

In [ ]:
X, Y = np.meshgrid(mags,shifts,indexing="ij")

fig = plt.figure()

gs = fig.add_gridspec(2,2, hspace=0,wspace=0)
ax2 = plt.subplot(gs.new_subplotspec((1, 0), rowspan=1))
ax3 = plt.subplot(gs.new_subplotspec((1, 1), rowspan=1),sharey=ax2)
ax0 = plt.subplot(gs.new_subplotspec((0, 0), rowspan=1),sharex=ax2)
ax1 = plt.subplot(gs.new_subplotspec((0, 1), rowspan=1),sharey=ax0,sharex=ax3)

col0 = ax0.pcolormesh(X, Y, fermi, cmap=plt.cm.inferno,shading=shade,snap=True)
ax0.set_title(r'$E_F$',fontsize=tfs, x=0.95, y=0.95, loc='left',color='white',backgroundcolor= 'black')
ax0.set_xlabel(r'$m$',fontsize=lfs)
ax0.set_ylabel(r'$\varphi$',fontsize=lfs)
cbar0 = plt.colorbar(col0,ax=ax1,shrink=0.95)
ax0.get_xaxis().set_visible(False)

col1 = ax1.pcolormesh(X, Y, ch_tau1tau2, vmin=-2.2,vmax=2.2, cmap=plt.cm.RdBu,shading=shade,snap=True)
ax1.set_title(r'$Ch_{t_1 t_2}$',fontsize=tfs, x=0.875, y=0.945, loc='left')
ax1.set_xlabel(r'$m$',fontsize=lfs)
ax1.set_ylabel(r'$\varphi$',fontsize=lfs)
cbar1 = plt.colorbar(col1,ax=ax3,shrink=0.95,extend='both')
cbar1.cmap.set_over('black')
cbar1.cmap.set_under('black')
ax1.get_xaxis().set_visible(False)
ax1.get_yaxis().set_visible(False)

col2 = ax2.pcolormesh(X, Y, ch_u1u2, vmin=-2.2,vmax=2.2, cmap=plt.cm.RdBu,shading=shade,snap=True)
ax2.set_title(r'$Ch_{u_1 u_2}$',fontsize=tfs, x=0.89, y=0.95, loc='left')
ax2.set_xlabel(r'$m$',fontsize=lfs)
ax2.set_ylabel(r'$\varphi$',fontsize=lfs)
# plt.colorbar(col2,ax=ax2,shrink=0.75)

col3 = ax3.pcolormesh(X, Y, ch_tau1tau2u1u2, vmin=-2.2,vmax=2.2, cmap=plt.cm.RdBu,shading=shade,snap=True)
ax3.set_title(r'$Ch_{t_1 t_2 u_1 u_2}$',fontsize=tfs, x=0.806, y=0.95, loc='left')
ax3.set_xlabel(r'$m$',fontsize=lfs)
ax3.set_ylabel(r'$\varphi$',fontsize=lfs)
ax3.get_yaxis().set_visible(False)
# plt.colorbar(col3,ax=ax3,shrink=0.75)

ax2.set_xticks(np.linspace(-2,2,8,endpoint=False))
ax2.xaxis.set_minor_locator(AutoMinorLocator())
ax3.set_xticks(np.linspace(-2,2,9))
ax3.xaxis.set_minor_locator(AutoMinorLocator())

labels = [r'${:.2f} \pi$'.format(n) for n in np.linspace(0,2,9)]

ax0.set_yticks(np.linspace(0,2*np.pi,9))
ax0.yaxis.set_minor_locator(AutoMinorLocator())
ax0.set_yticklabels(labels,fontsize=fs)

labels = [r'${:.2f} \pi$'.format(n) for n in np.linspace(0,2,8,endpoint=False)]
ax2.set_yticks(np.linspace(0,2*np.pi,8,endpoint=False))
ax2.yaxis.set_minor_locator(AutoMinorLocator())
ax2.set_yticklabels(labels,fontsize=fs)

for ax in fig.get_axes()[:-2]:
    ax.tick_params(axis='both', which='major', labelsize=fs)
    
    for s in winding_boundary(net_mag1): ax.plot(net_mag1,s,c='k',linewidth=1.0);
    for s in winding_boundary(net_mag2): ax.plot(net_mag2,s,c='k',linewidth=1.0);

fig.tight_layout()
# plt.savefig("./Plots/ChernDiagram2x2.png", dpi=300, bbox_inches = 'tight')
# plt.savefig("./Plots/lowres/ChernDiagram2x2.png", dpi=100, bbox_inches = 'tight')